In [19]:
# libraries
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os
import cv2
from sklearn.model_selection import train_test_split

In [20]:
!wget https://get.station307.com/5DWErPoEjHd/data.zip

--2020-08-21 04:03:22--  https://get.station307.com/5DWErPoEjHd/data.zip
Resolving get.station307.com (get.station307.com)... 37.139.6.27
Connecting to get.station307.com (get.station307.com)|37.139.6.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503919476 (481M) [application/x-zip-compressed]
Saving to: ‘data.zip.1’

data.zip.1          100%[===================>] 480.57M   502KB/s    in 13m 2s  

2020-08-21 04:16:25 (629 KB/s) - ‘data.zip.1’ saved [503919476/503919476]



In [21]:
# constants
img_dir = 'data/images'
data = pd.read_csv('legend.csv')
images = data["image"]
y = data['emotion']
y = [i.lower() for i in y]

In [22]:
!unzip data.zip

Archive:  data.zip
replace data/facial_expressions/.git/config? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/facial_expressions/.git/config  
replace data/facial_expressions/.git/description? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace data/facial_expressions/.git/description? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [23]:
# image resizer
def image_reader(location):
    img = cv2.imread(location, 0)
    img = cv2.resize(img, (50, 50))
    return img

In [24]:
emotion = set(y)
emotion = list(emotion)
print(emotion)
y = [emotion.index(i) for i in y]

['disgust', 'happiness', 'contempt', 'fear', 'sadness', 'anger', 'neutral', 'surprise']


In [25]:
# input data
x = []
for img in range(len(images)):
    location = os.path.join(img_dir, images[img])
    x.append(image_reader(location))
x = np.array(x)
x = x.reshape(13690, 50, 50, 1)
y = np.array(y)
print('Completed')

Completed


In [26]:
# model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu',
                        input_shape=(50, 50, 1)))
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(8, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
x1, x2, y1, y2 = train_test_split(x, y, test_size = 0.3)

In [28]:
epochs = 100
batch_size = 128

history = model.fit(x1, y1, epochs=epochs,
          validation_data=(x2, y2), shuffle=True, verbose=1)

Epoch 1/100
300/300 [==============================] - 3s 11ms/step - loss: 1.0508 - accuracy: 0.5543 - val_loss: 0.8221 - val_accuracy: 0.7129
Epoch 2/100
300/300 [==============================] - 3s 10ms/step - loss: 0.7711 - accuracy: 0.7425 - val_loss: 0.7521 - val_accuracy: 0.7412
Epoch 3/100
300/300 [==============================] - 3s 10ms/step - loss: 0.6634 - accuracy: 0.7815 - val_loss: 0.6151 - val_accuracy: 0.7955
Epoch 4/100
300/300 [==============================] - 3s 10ms/step - loss: 0.5821 - accuracy: 0.8125 - val_loss: 0.5512 - val_accuracy: 0.8191
Epoch 5/100
300/300 [==============================] - 3s 10ms/step - loss: 0.5349 - accuracy: 0.8257 - val_loss: 0.5277 - val_accuracy: 0.8283
Epoch 6/100
300/300 [==============================] - 3s 10ms/step - loss: 0.4977 - accuracy: 0.8368 - val_loss: 0.5171 - val_accuracy: 0.8310
Epoch 7/100
300/300 [==============================] - 3s 10ms/step - loss: 0.4543 - accuracy: 0.8490 - val_loss: 0.5252 - val_accuracy:

In [29]:
test_data = cv2.imread('test.jpg', 0)
test_data = cv2.resize(test_data, (60, 50))
test_data.resize(1, 50, 50, 1)

In [30]:
predictions = model.predict(test_data)

In [31]:
emotion[predictions.argmax()]

'happiness'